<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed and computed data in the dataSource, the import scopes with for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

In [0]:
#!eval-notebook "../CloseImportTemplate"

# Args

In [0]:
var reportingNode = "CH";
var year = 2020;
var month = 12;
var importFormat = ImportFormats.Cashflow;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

# EstimateType to load

In [0]:
var estimateTypes = (await DataSource.Query<EstimateType>().ToArrayAsync());

In [0]:
var estimateTypesByImportFormat = new Dictionary<string, IEnumerable<string>>(); 
estimateTypesByImportFormat[ImportFormats.Actual] = estimateTypes.Where(et => (et.InputSource == (InputSource)2 || et.InputSource == (InputSource)3) && 
                                                        et.SystemName != EstimateTypes.APA)
                                            .Select(x => x.SystemName);
estimateTypesByImportFormat[ImportFormats.Cashflow] = estimateTypes.Where(et => et.InputSource == (InputSource)4 && 
                                                        et.SystemName != EstimateTypes.BEPA && et.SystemName != EstimateTypes.F)
                                            .Select(x => x.SystemName);
estimateTypesByImportFormat[ImportFormats.Opening] = estimateTypes.Where(et => (et.InputSource == (InputSource)3 || et.InputSource == (InputSource)7) && 
                                                        et.SystemName != "PL")
                                            .Select(x => x.SystemName);

estimateTypesByImportFormat

# Load variables to workspace

In [0]:
DataSource.Partition.GetCurrent()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args);
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args);

In [0]:
switch (args.ImportFormat)
{ 
    case ImportFormats.Actual : 
        await Workspace.UpdateAsync<IfrsVariable>(await DataSource.Query<IfrsVariable>().Where(x => estimateTypesByImportFormat[ImportFormats.Actual].Contains(x.EstimateType)).ToArrayAsync());
        break;
    case ImportFormats.Cashflow : 
        await Workspace.UpdateAsync<RawVariable>(await DataSource.Query<RawVariable>().Where(x => estimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType)).ToArrayAsync());
        break;
    default : 
        await Workspace.UpdateAsync<IfrsVariable>(await DataSource.Query<IfrsVariable>().Where(x => estimateTypesByImportFormat[ImportFormats.Opening].Contains(x.EstimateType)).ToArrayAsync());
        break;
}

# Set up a copy of the DataSource

In [0]:
var customDataSource = Workspace.CreateNew();

In [0]:
customDataSource.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, customDataSource, Workspace);
await storage.InitializeAsync();

# Create universe and Identities

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [0]:
var pvLocked = universe.GetScopes<PvLocked>(identities)
                .SelectMany(x => x.PresentValues)
                .Select(x => new {Value= x.Value, 
                                  Id = x.Identity.Id, 
                                  AmoutType = x.Identity.AmountType,
                                  EstimateType = x.Identity.EstimateType,
                                  AccidentYear = x.Identity.AccidentYear});
pvLocked